In [175]:
# -m pip install plotly
# -m pip install scipy
# -m pip install nbformat
# -m pip install --upgrade nbformat

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objs as go
from IPython.display import HTML
from plotly.offline import init_notebook_mode, iplot
import scipy
import os

init_notebook_mode(connected=True)

In [176]:
# Pokemon type colors.
colors = {
    "bug": "#A6B91A",
    "dark": "#705746",
    "dragon": "#6F35FC",
    "electric": "#F7D02C",
    "fairy": "#D685AD",
    "fighting": "#C22E28",
    "fire": "#EE8130",
    "flying": "#A98FF3",
    "ghost": "#735797",
    "grass": "#7AC74C",
    "ground": "#E2BF65",
    "ice": "#96D9D6",
    "normal": "#A8A77A",
    "poison": "#A33EA1",
    "psychic": "#F95587",
    "rock": "#B6A136",
    "steel": "#B7B7CE",
    "water": "#6390F0",
}

In [177]:
# Import the CSV file.
pokemon = pd.read_csv("pokemon.csv")

pokemon.head()
pokemon.hp

In [178]:
# HP Single Displot
hp_distplot = ff.create_distplot([pokemon.hp], ['HP'], bin_size=5)
iplot(hp_distplot, filename='HP Distplot')

In [179]:
# Attack & Defense, Double Displot 
attack_defense_distplot = ff.create_distplot([pokemon.attack, pokemon.defense], ['Attack', 'Defense'], bin_size=5)
iplot(attack_defense_distplot, filename='Attack/Defense Distplot')

In [180]:
def polar_pokemon_stats(pkmn_name):
    pkmn = pokemon[pokemon.name == pkmn_name]
    obj = go.Scatterpolar(
        r=[
            pkmn['attack'].values[0],
            pkmn['defense'].values[0],
            pkmn['sp_attack'].values[0],
            pkmn['sp_defense'].values[0],
            pkmn['speed'].values[0],
            pkmn['hp'].values[0]
        ],
        theta=[
            'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'HP'
        ],
        fill='toself',
        marker=dict(
            color=colors[pkmn['type1'].values[0]]
        ),
        name=pkmn['name'].values[0]
    )

    return obj

In [181]:
def plot_single_pokemon(name):
    layout = go.Layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 250]
            )
        ),
        showlegend=False,
        title="Stats of {}".format(name)
    )

    pokemon_figure = go.Figure(data=[polar_pokemon_stats(name)], layout=layout)
    iplot(pokemon_figure, filename='Single Pokemon')

name = 'Charmander'
plot_single_pokemon(name)

In [182]:
# Comparing stats of 2 different pokemons
pkmn_1_name = 'Kyogre'
pkmn_2_name = 'Entei'

layout = go.Layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 250]
        )
    ),
    showlegend=True,
    title="{} vs. {}".format(pkmn_1_name, pkmn_2_name)
)

pkmn_1 = polar_pokemon_stats(pkmn_1_name)
pkmn_2 = polar_pokemon_stats(pkmn_2_name)

compare_2_pokemon = go.Figure(data=[pkmn_1, pkmn_2], layout=layout)
iplot(compare_2_pokemon, filename='Compare 2 Pokemon')

In [183]:
def get_correlation_stats(stat1, stat2, stat3='speed'):
    data = go.Scatter(
        x=pokemon[stat1],
        y=pokemon[stat2],
        mode='markers',
        marker=dict(
            size=10,
            color=pokemon[stat3],
            colorscale='Viridis',
            showscale=True
        ),
        text=pokemon['name']
    )

    layout = go.Layout(
        xaxis=dict(
            title=stat1
        ),
        yaxis=dict(
            title=stat2
        ),
        showlegend=False,
        title="Scatter plot of {} and {}, colored on {}".format(stat1, stat2, stat3)
    )

    correlation_stats = go.Figure(data=[data], layout=layout)
    return correlation_stats


iplot(get_correlation_stats('hp', 'defense'), filename='Stats Correlation')

In [184]:
# Visualizing number of pokemon per type across all generation.
types = (pokemon.groupby(['type1'])['pokedex_number'].count())
types_name = list(types.keys())

data = go.Bar(
    x=types_name,
    y=types.values,
    marker=dict(
        color=list(colors.values())
    ),
    name="{}".format(types_name)
)

layout = go.Layout(
    title='Types',
    xaxis=dict(
        title='Type'
    ),
    yaxis=dict(
        title='Number of Pokemon'
    )
)

fig = go.Figure(data=[data], layout=layout)
iplot(fig, filename='Types')

In [185]:
# Visualizing the trend of stats by type and generation.
def stats_by(classifier):
    data = []
    stats_names = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']
    stats = pokemon.groupby(classifier)[stats_names].mean().reset_index()
    for stat in stats_names:
        stat_line = go.Scatter(
            x=stats[classifier],
            y=stats[stat],
            name=stat,
            line=dict(
                width=3,
            ),
        )

        data.append(stat_line)

    layout = go.Layout(
        title='Trend of stats by {}'.format(classifier),
        xaxis=dict(title=classifier),
        yaxis=dict(title='Values')
    )

    trend = go.Figure(data=data, layout=layout)
    iplot(trend, filename='trend')


stats_by('type1')

In [186]:
# Code sourced from: https://www.kaggle.com/code/lakshyaag/data-visualization-pokemon-data